# Install QBAF-Py

In [ ]:
%%capture
!apt install libgraphviz-dev # Necessary to install the visualizer in google colab

In [ ]:
%%capture
# Install QBAF-Py and the visualizer dependency
!git clone https://github.com/TimKam/Quantitative-Bipolar-Argumentation
!pip install ./Quantitative-Bipolar-Argumentation[visualizer]

In [ ]:
# get CPU spec
!cat /proc/cpuinfo

# Execution times

## Helper code

In [ ]:
# Imports
import time
import random

from statistics import mean

from qbaf import QBAFramework, QBAFARelations
from qbaf_visualizer.Visualizer import visualize

In [ ]:
def QBAF_relations(qbaf:QBAFramework) -> list:
  return list(qbaf.attack_relations.relations) + list(qbaf.support_relations.relations)

def QBAF_contains_relation(qbaf:QBAFramework, agent, patient) -> bool:
  return qbaf.contains_attack_relation(agent, patient) or qbaf.contains_support_relation(agent, patient)

In [ ]:
# Pop random item from a list
def pop_random(l:list):
  assert len(l) > 0
  index = random.randint(0, len(l) - 1)
  item = l[index]
  del l[index]
  return item

# Get random item from list
def get_random(l:list):
  assert len(l) > 0
  index = random.randint(0, len(l) - 1)
  item = l[index]
  return item

In [ ]:
# Modify a qbaf randomly
def modify_QBAF_random(qbaf:QBAFramework, n_modify_arguments:int, n_remove_relations:int, n_add_relations:int) -> QBAFramework:
  assert n_modify_arguments >= 0 and n_modify_arguments <= len(qbaf.arguments)
  assert n_remove_relations >= 0 and n_remove_relations <= (len(qbaf.attack_relations) + len(qbaf.support_relations))
  assert n_add_relations >= 0

  # Modify arguments
  arguments = list(qbaf.arguments)
  for _ in range(n_modify_arguments):
    arg = pop_random(arguments)
    i_strength = random.uniform(0, 1)
    qbaf.modify_initial_strength(arg, i_strength)

  # Remove relations
  relations = QBAF_relations(qbaf)
  for _ in range(n_remove_relations):
    agent, patient = pop_random(relations)
    qbaf.remove_attack_relation(agent, patient)
    qbaf.remove_support_relation(agent, patient)

  # Add relations
  added_relations = 0
  while added_relations < n_add_relations:
    arguments = list(qbaf.arguments)
    arg1 = pop_random(arguments)
    arg2 = get_random(arguments)
    if QBAF_contains_relation(qbaf, arg1, arg2):
      continue
    if random.randint(0, 1) == 1:
      qbaf.add_attack_relation(arg1, arg2)
    else:
      qbaf.add_support_relation(arg1, arg2)
    added_relations += 1

    # Check that the QBAF is acyclic
    if not qbaf.isacyclic():
      qbaf.remove_attack_relation(arg1, arg2)
      qbaf.remove_support_relation(arg1, arg2)
      added_relations -= 1

  return qbaf

# Modify a qbaf randomly, for change explanations
def modify_QBAF_random_update(qbaf:QBAFramework, n_modify_arguments:int) -> QBAFramework:
  assert n_modify_arguments >= 0 and n_modify_arguments <= len(qbaf.arguments)

  qbaf_backup = qbaf.copy()
  # Modify arguments
  arguments = list(qbaf.arguments)
  for i in range(n_modify_arguments):
    if i <= i/5: # remove argument
      arg = pop_random(arguments)
      relations = QBAF_relations(qbaf)
      for agent, patient in relations:
        if agent == arg or patient == arg:
          qbaf.remove_attack_relation(agent, patient)
          qbaf.remove_support_relation(agent, patient)
      qbaf.remove_argument(arg)
    elif i <= 2 * (i/5): # change strength
      arg = pop_random(arguments)
      i_strength = random.uniform(0, 1)
      qbaf.modify_initial_strength(arg, i_strength)
    elif i <= 3 * (i/5):
      # remove relation
      relations = QBAF_relations(qbaf)
      agent, patient = pop_random(relations)
      qbaf.remove_attack_relation(agent, patient)
      qbaf.remove_support_relation(agent, patient)
    elif i <= 3 * (i/5): # add relation
      arguments = list(qbaf.arguments)
      arg1 = pop_random(arguments)
      arg2 = get_random(arguments)
      if QBAF_contains_relation(qbaf, arg1, arg2):
        continue
      if random.randint(0, 1) == 1:
        qbaf.add_attack_relation(arg1, arg2)
      else:
        qbaf.add_support_relation(arg1, arg2)
    else: # add argument
        qbaf.add_argument(f'a{i}', random.uniform(0, 1))
        arguments = list(qbaf.arguments)
        # add on average 1 relation, outgoing
        for _ in range(random.choice([0, 1, 2])):
          relations = QBAF_relations(qbaf)
          arg2 = pop_random(arguments)
          if random.randint(0, 1) == 1:
            qbaf.add_attack_relation(f'a{i}', arg2)
          else:
            qbaf.add_support_relation(f'a{i}', arg2)
        # add on average 1/5 relation, incoming
        for _ in range(random.choice([0, 0, 0, 0, 1])):
          relations = QBAF_relations(qbaf)
          arg2 = pop_random(arguments)
          if random.randint(0,1) == 1:
            qbaf.add_attack_relation(arg2, f'a{i}')
          else:
            qbaf.add_support_relation(arg2, f'a{i}')

  # Check that the QBAF is acyclic
  if not qbaf.isacyclic():
    return modify_QBAF_random_update(qbaf_backup, n_modify_arguments)

  return qbaf

# Create a random QBAF
def QBAF_random(n_arguments:int, n_relations:int, semantics="QuadraticEnergy_model") -> QBAFramework:
  assert n_arguments >= 0 and n_relations >= 0
  # Create arguments as strings of the numbers 1..n_arguments
  args = [str(n+1) for n in range(n_arguments)]
  # Generate random strengths in range [0,1] for each argument
  i_strengths = [random.uniform(0, 1) for _ in range(n_arguments)]
  # Create the QBAF without relations
  qbaf = QBAFramework(args, i_strengths, [], [], semantics=semantics)
  # Add the relations
  modify_QBAF_random(qbaf, n_modify_arguments=0, n_remove_relations=0, n_add_relations=n_relations)
  # Return the QBAF
  return qbaf

In [ ]:
def random_inconsistent_arguments(qbf1:QBAFramework, qbf2:QBAFramework) -> tuple:
  arguments = list(qbf1.arguments.intersection(qbf2.arguments))
  s_inconstent_args = [(arg1, arg2) for i, arg1 in enumerate(arguments) for arg2 in arguments[i:] if not qbf1.are_strength_consistent(qbf2, arg1, arg2)]
  if len(s_inconstent_args) > 0:
    return get_random(s_inconstent_args)
  return None

## Generating random QBAFs

In [ ]:
def random_QBAFs(n_arguments:int,            # number of arguments
                 prop_relations=3,           # number of relations in proportion to n_arguments (generated QBAF)
                 semantics="QuadraticEnergy_model",
                 size=1,                     # number of generated QBAFs
                 seed=1234
                 ) -> list:
  if seed is not None:
    random.seed(seed)

  n_relations = int(n_arguments * prop_relations)

  result = []
  while len(result) < size:
    qbaf = QBAF_random(n_arguments, n_relations, semantics=semantics) # generated QBAF
    result.append(qbaf)  # generated_QBAF

  return result # Return a list of QBAF:QBAFramework

In [ ]:
# generate random QBAFs with its updated version
def generate_random_QBAFs(n_arguments:int,            # number of arguments (generated QBAF)
                          prop_relations=1.1,           # number of relations in proportion to n_arguments (generated QBAF)
                          prop_modify_arguments=0.2,  # number of modified arguments in proportion to n_arguments (updated QBAF)
                          semantics="QuadraticEnergy_model",
                          size=1,                     # number of pairs generated, updated QBAFs
                          seed=1234
                          ) -> list:
  if seed is not None:
    random.seed(seed)

  n_relations = int(n_arguments * prop_relations)
  n_modify_arguments = int(n_arguments * prop_modify_arguments)

  result = []
  while len(result) < size:
    qbaf1 = QBAF_random(n_arguments, n_relations, semantics=semantics) # generated QBAF
    qbaf2 = qbaf1.copy()
    qbaf2 = modify_QBAF_random_update(qbaf2, n_modify_arguments)
    s_inconsistent_args = random_inconsistent_arguments(qbaf1, qbaf2)
    if s_inconsistent_args is None:
      continue
    result.append((qbaf1, qbaf2, s_inconsistent_args))  # generated_QBAF, updated_QBAF, (arg1, arg2)

  return result # Return a list of tuples (generated_QBAF:QBAFramework, updated_QBAF:QBAFramework, pair_strength_inconsistent_arguments:tuple)

## Testing execution times

### Calculate final strengths

In [ ]:
n = 50 # Number of QBAFs generated each time

n_arguments_list = []
results_list = []
for i in range(100):
  n_arguments = (i + 1) * 10
  times = []
  for qbaf in random_QBAFs(n_arguments=n_arguments, semantics="QuadraticEnergy_model", size=n, seed=1234):
    start_time = time.time()
    f_strengths = qbaf.final_strengths
    end_time = time.time()
    times.append(end_time - start_time)

  avg_time = mean(times)
  n_arguments_list.append(n_arguments)
  results_list.append(avg_time)
  print(f'For {n_arguments} arguments, the average time in {n} QBAFs is {avg_time} seconds.')



In [ ]:
import matplotlib.pyplot as plt
plot = plt.plot(n_arguments_list, results_list, color='black')
plt.ylabel('Average final strength computation time (in seconds)')
plt.xlabel('Number of arguments')

### Finding Strength Inconsistency Explanations

In [ ]:
from statistics import median

def run_experiment(change_amount, n, maximum, offset=5):
  # SSI explanations
  for i in range(maximum-4):
    n_arguments = i + offset
    ssi_times = []
    csi_times = []
    nsi_times = []
    for qbaf1, qbaf2, (arg1, arg2) in generate_random_QBAFs(n_arguments=n_arguments, semantics="QuadraticEnergy_model", size=n, seed=123, prop_modify_arguments=change_amount):
      start_time = time.time()
      explanations = qbaf1.minimalSSIExplanations(qbaf2, arg1, arg2)
      end_time = time.time()
      ssi_times.append(end_time - start_time)
      start_time = time.time()
      explanations = qbaf1.minimalCSIExplanations(qbaf2, arg1, arg2)
      end_time = time.time()
      csi_times.append(end_time - start_time)
      start_time = time.time()
      explanations = qbaf1.minimalNSIExplanations(qbaf2, arg1, arg2)
      end_time = time.time()
      nsi_times.append(end_time - start_time)

    n_arguments_list.append(n_arguments)

    ssi_avg_time = mean(ssi_times)
    ssi_results_list.append(ssi_avg_time)
    max_ssi_avg_time = max(ssi_times)
    max_ssi_results_list.append(max_ssi_avg_time)
    med_ssi_avg_time = median(ssi_times)
    med_ssi_results_list.append(med_ssi_avg_time)
    print(f'SSI explanations: for {n_arguments} arguments, the average time in {n} QBAFs is {ssi_avg_time} seconds.')
    csi_avg_time = mean(csi_times)
    csi_results_list.append(csi_avg_time)
    max_csi_avg_time = max(csi_times)
    max_csi_results_list.append(max_csi_avg_time)
    med_csi_avg_time = median(csi_times)
    med_csi_results_list.append(med_csi_avg_time)
    print(f'CSI explanations: for {n_arguments} arguments, the average time in {n} QBAFs is {csi_avg_time} seconds.')
    nsi_avg_time = mean(nsi_times)
    nsi_results_list.append(nsi_avg_time)
    max_nsi_avg_time = max(nsi_times)
    max_nsi_results_list.append(max_nsi_avg_time)
    med_nsi_avg_time = median(nsi_times)
    med_nsi_results_list.append(med_nsi_avg_time)
    print(f'NSI explanations: for {n_arguments} arguments, the average time in {n} QBAFs is {nsi_avg_time} seconds.')


In [ ]:
n_arguments_list = []
ssi_results_list = []
csi_results_list = []
nsi_results_list = []
max_ssi_results_list = []
max_csi_results_list = []
max_nsi_results_list = []
med_ssi_results_list = []
med_csi_results_list = []
med_nsi_results_list = []
run_experiment(0.2, 50, 100)

In [ ]:
import matplotlib.pyplot as plt
plot = plt.plot(n_arguments_list, ssi_results_list, label ='SSI', color='black')
plot = plt.plot(n_arguments_list, csi_results_list, '-.', label ='CSI', color='black')
plot = plt.plot(n_arguments_list, nsi_results_list, '--', label ='NSI', color='black')
plt.legend()
plt.ylabel('Average explanation computation time (in seconds)')
plt.xlabel('Number of arguments')

In [ ]:
plot = plt.plot(n_arguments_list, max_ssi_results_list, label ='SSI', color='black')
plot = plt.plot(n_arguments_list, max_csi_results_list, '-.', label ='CSI', color='black')
plot = plt.plot(n_arguments_list, max_nsi_results_list, '--', label ='NSI', color='black')
plt.legend()
plt.ylabel('Maximal explanation computation time (in seconds)')
plt.xlabel('Number of arguments')

In [ ]:
plot = plt.plot(n_arguments_list, med_ssi_results_list, label ='SSI', color='black')
plot = plt.plot(n_arguments_list, med_csi_results_list, '-.', label ='CSI', color='black')
plot = plt.plot(n_arguments_list, med_nsi_results_list, '--', label ='NSI', color='black')
plt.legend()
plt.ylabel('Median explanation computation time (in seconds)')
plt.xlabel('Number of arguments')